In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

df = pd.read_csv('data/fit.csv')
train = df[0: df.shape[0] // 10 * 8]
validation = df[df.shape[0] // 10 * 8: df.shape[0] // 10 * 9]
test = df[df.shape[0] // 10 * 9:]

# SVD
col_names = ['user_id', 'item_id', 'rating']
reader = Reader(rating_scale=(2, 10))
data = Dataset.load_from_df(train[col_names], reader)
data = data.build_full_train_pairsset()
algo = SVD()
algo.fit(data)
svds = []
for user_id, book_id in zip(train['user_id'], train['item_id']):
    svds.append(algo.predict(user_id, book_id).est)
train['SVD'] = svds

one_hot_rf = pd.get_dummies(train['rented for'])
one_hot_bt = pd.get_dummies(train['body type'])
one_hot_cat = pd.get_dummies(train['category'])

train = train.drop('rented for',axis = 1)
train = train.drop('body type',axis = 1)
train = train.drop('category',axis = 1)

train = train.drop('review date',axis = 1)
train = train.drop('fit', axis = 1)

ratings = train['rating']
train = train.drop('rating', axis = 1)

train = train.join(one_hot_rf)
train = train.join(one_hot_bt)
train = train.join(one_hot_cat)

reg = LinearRegression().fit(train, ratings)